In [1]:
# Import modules
from pymongo import MongoClient
import pandas as pd
import json as js
import matplotlib.pyplot as plt

In [2]:
client = MongoClient("mongodb://localhost/companies")
db = client.get_database()

In [3]:
df = pd.DataFrame(list(db["companies"].find({'founded_year': {'$gte':2003}})))
df.shape

(9297, 42)

In [4]:
# DataFrame with companies founded before 2003
df_oldestcomp = pd.DataFrame(list(db['companies'].find({'founded_year': {'$lt': 2003}}, {"name":1,"offices":1,"category_code":1})))
df_oldestcomp.shape

(3839, 4)

In [5]:
# Total companies with declared founded year = 13.136.
# Companies with founded year null = 5665

In [6]:
# Look for different currencies on raised_currency_code for a future money exchange

currency = list(db['companies'].distinct('funding_rounds.raised_currency_code'))
currency

[None, 'CAD', 'EUR', 'GBP', 'JPY', 'SEK', 'USD']

In [7]:
#To filter for tech startups that raised more than $1M, first know which types of startups are in the database

startups_type = list(db['companies'].distinct('category_code'))


In [8]:
# Data Frame with Tech Statups founded after 2002 that raised $1M on a funding round. 6977
df = pd.DataFrame(list(db['companies'].find({'$and': [{'founded_year': {'$gte':2003}}, {'category_code': {'$in': ['advertising', 'analytics', 'consulting', 'design', 'ecommerce', 'games_video', 'hardware', 'mobile', 'nanotech', 'network_hosting', 'software', 'web']}}, {'funding_rounds.raised_amount': {'$gte': 1000000}}]}, {"name":1,"offices":1,"category_code":1})))
df.shape

(1468, 4)

In [9]:
# Create a DataFrame with Starbucks location to add to paint markers on the offices map
df_st = pd.read_csv('input/directory.csv', engine='python')
df_st = df_st.drop(columns=['Ownership Type', 'Store Number', 'Ownership Type', 'Postcode', 'Phone Number', 'Timezone'])
df_st = df_st.dropna(subset= ['Latitude', 'Longitude'])

In [10]:
# Explode offices to have each office (dictionary, part of a list) on a row
df = df.explode('offices')

# Expand offices info into different columns
dfOffices = df[["offices"]].apply(lambda r: r.offices, result_type="expand", axis=1)

#Concanete both DataFrames
cleanData = pd.concat([df,dfOffices], axis=1)

In [11]:
#Same process for DataFrame of companies founded before 2003
df_oldestcomp = df_oldestcomp.explode('offices')
df_oldestcompOffices = df_oldestcomp[['offices']].apply(lambda r: r.offices, result_type='expand', axis=1)
oldestOffices = pd.concat([df_oldestcomp,df_oldestcompOffices], axis=1)

In [12]:
# Cleaning offices DataFrame dropping unnecessary columns and null info for latitude and longitude
cleanData = cleanData.drop(columns= ['_id', 'offices'])
cleanData = cleanData.dropna(subset= ['latitude', 'longitude'])

In [13]:
# Cleaning oldest offices DataFrame dropping unnecessary columns and null info for latitude and longitude
oldestOffices = oldestOffices.drop(columns= ['_id', 'offices'])
oldestOffices = oldestOffices.dropna(subset= ['latitude', 'longitude'])

In [14]:
#Create a function to apply to latitude and longitude series to create a new column with a JSON 'point'
#Why I need this column?
def asGeoJSON(lat,lng):
    lat = float(lat)
    lng = float(lng)
    return {
        "type":"Point",
        "coordinates":[lng,lat]
    }
        

cleanData["location"] = cleanData[["latitude","longitude"]].apply(lambda x:asGeoJSON(x.latitude,x.longitude), axis=1)

oldestOffices["location"] = oldestOffices[["latitude","longitude"]].apply(lambda x:asGeoJSON(x.latitude,x.longitude), axis=1)

df_st['location'] = df_st[['Latitude', 'Longitude']].apply(lambda x:asGeoJSON(x.Latitude,x.Longitude), axis=1)

In [15]:
# Select 3 cities from the original DataFrame based on the number of value_counts of startups in the original DataFrame 
cleanData['city'].value_counts().head(3)

# Cities to compare: San Francisco, New York, Mountain View
# Filter both DataFrames where cities are previous ones.

cleanData = cleanData[(cleanData['city'] == 'San Francisco') | (cleanData['city'] == 'New York') | (cleanData['city'] == 'Mountain View')]

df_st = df_st[(df_st['City'] == 'San Francisco') | (df_st['City'] == 'New York') | (df_st['City'] == 'Mountain View')]

oldestOffices = oldestOffices[(oldestOffices['city'] == 'San Francisco') | (oldestOffices['city'] == 'New York') | (oldestOffices['city'] == 'Mountain View')]

In [16]:
# Cleaning DataFrames
cleanData = cleanData.drop(columns=['latitude', 'longitude'])
oldestOffices = oldestOffices.drop(columns=[ 'latitude', 'longitude'])
df_st = df_st.drop(columns=['Latitude', 'Longitude'])

In [17]:
# Exporting data for MongoDB. 
cleanData.to_json("output/cleanOffices.json", orient="records")
oldestOffices.to_json("output/oldestOffices.json", orient="records")
df_st.to_json("output/starbucks.json", orient="records")